In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina' 
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import gseapy as gp
from matplotlib import pyplot as plt
from tqdm import tqdm
import pickle
from gseapy import barplot, dotplot

In [ ]:
inputExcelPath = "sigX_data.xlsx"

rawUpDF = pd.read_excel(inputExcelPath, sheet_name = "Increased in PAOSX")[["Locus tag", "Gene name", "Fold change", "T statistic", "P-value"]]
rawDownDF = pd.read_excel(inputExcelPath, sheet_name = "Decreased in PAOSX")[["Locus tag", "Gene name", "Fold change", "T statistic", "P-value"]]

backgroundPAO1 = pd.read_csv("PAO1_Conversion_df.csv")["Locus Tag"].to_list()

In [ ]:
with open("pae_kegg_pathways.gmt", "r") as f:
    paths = f.read()
paths = paths.split(sep = "\n")
paths

def filterPathwayLength(inputPaths, minLength):
    outPaths = []
    for path in inputPaths:
        spath = path.split("\t")[2:]
        if len(spath) >= minLength:
            outPaths.append(path)
    with open("tmp.gmt", "w") as w:
        for path in outPaths:
            w.write(path + "\n")

filterPathwayLength(paths, 10)

In [ ]:
enr = gp.enrich(
    gene_list = rawUpDF["Locus tag"].to_list(),
    gene_sets = "tmp.gmt",
    background = backgroundPAO1,
    outdir = None,
    verbose = True
    )
# gseapy is just a wrapper for Enrichr, which yields this output:
# p = p-value computed using the Fisher exact test (Hypergeometric test)
# z = z-score (Odds Ratio)
# combine score = - log(p)·z        

enrOut = enr.results

with open('pae_descriptions.pkl', 'rb') as f:
    pathway2desc = pickle.load(f)
    
enrOut['PathName'] = enrOut['Term'].map(pathway2desc)

In [ ]:
sigOut = enrOut.loc[enrOut["Adjusted P-value"] <= 0.05].sort_values(by = "Adjusted P-value")
sigOut["Term"] = sigOut["PathName"]
sigOut["Combined Score"] = sigOut["Combined Score"].apply(lambda x: np.log10(x) if x > 0 else 0)

In [ ]:
dotInputDf = sigOut.copy().head(20)

ax = dotplot(dotInputDf,
             hue = "Adjusted P-value",
             y_order = dotInputDf["Term"].tolist(),
             x = None,
             top_term = 20,
             figsize = (5,15),
             xticklabels_rot = 45,
             show_ring = True,
             marker = 'o'
             )
ax.set_xlabel("log10(Combined Score)")  # <-- Add this line


In [ ]:
ax = barplot(sigOut,
             column = "Adjusted P-value",
             top_term = 10,
             figsize=(10,10),
             hue = "Overlap"
             )